In [ ]:
import os
import pyemu
import flopy
from pathlib import Path
import shutil
import pandas as pd

In [ ]:
org_ws = Path('../MV_Model_Files/')
new_ws = 'simple_ies'
exe_dir = Path('../bin/mac/')

In [ ]:
if os.path.exists(new_ws):
    shutil.rmtree(new_ws)
shutil.copytree(org_ws, new_ws)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws = new_ws)

In [ ]:
shutil.copy2(os.path.join(exe_dir,'mf6'),os.path.join(new_ws,'mf6'))

In [ ]:
pyemu.os_utils.run('mf6',cwd=new_ws)

In [ ]:
#need a spatial reference to use pilot points. this means xoff, yoff, rotation, epsg
m=sim.get_model()
m.modelgrid

In [ ]:
pf = pyemu.utils.PstFrom(original_d=new_ws,new_d='template',remove_existing=True,start_datetime="1-1-2025",spatial_reference=m.modelgrid)

In [ ]:
?pf.add_parameters

In [ ]:
k_pars = [f for f in os.listdir(new_ws) if 'k_' in f and f.endswith('.ref')]
k_pars
for f in k_pars:
    pf.add_parameters(f,par_type='constant',upper_bound=20,lower_bound=0.05)

In [ ]:
#for pilot points, we first need to build a variogram, which describes the underlying spatial correlation structure of the variable
pp_v = pyemu.geostats.ExpVario(contribution=1.0, a=5000) #rule of thumb is that range should be atleast 2x pp spacing (delr= 500
pp_gs = pyemu.geostats.GeoStruct(variograms=pp_v)
ax=pp_gs.plot()
ax.axvline(x=500*5*2)

In [ ]:
for f in k_pars:
    pf.add_parameters(f,par_type='pilotpoint',upper_bound=20,lower_bound=0.05,pp_space=5,geostruct=pp_gs)

In [ ]:
?pf.add_observations

In [ ]:
obs_csvs = [f for f in os.listdir(new_ws) if f.endswith('.csv')]
for f in obs_csvs:
    df = pd.read_csv(os.path.join(new_ws,f))
    pf.add_observations(f,index_cols='time',use_cols=df.columns.tolist()[1:])

In [ ]:
pf.mod_sys_cmds.append('mf6')

In [ ]:
pf.build_pst(filename=os.path.join(pf.new_d,'at.pst'),version=2)

In [ ]:
pst = pyemu.Pst(filename=os.path.join(pf.new_d,'at.pst'))
obs = pst.observation_data
obs

In [ ]:
#zero everything out for starters
obs.weight = 0

In [ ]:
#we need to tell PEST what are the observed values and their weights, to calc the objective function
truth = pd.read_csv(os.path.join(new_ws,'obsvalues.dat'),delim_whitespace=True)
truth

In [ ]:
for i,stuff in truth.iterrows():
    obs.loc[obs.obsnme.str.contains(stuff.obsnme),'obsval'] = stuff.obsval
    obs.loc[obs.obsnme.str.contains(stuff.obsnme),'weight'] = stuff.weight
    obs.loc[obs.obsnme.str.contains(stuff.obsnme),'obgnme'] = stuff.obgnme

In [ ]:
pst.observation_data = obs
pst.write(os.path.join(pf.new_d,'at.pst'),version=2)

In [ ]:
pyemu.os_utils.run('pestpp-ies at.pst',cwd='template')

In [ ]:
phi= pst.phi
phi